In [ ]:
from mlwpy import *
import pymc3 as pm

# getting really hard to convince toolkits to be less verbose
import logging
pymc3_log = logging.getLogger('pymc3')
pymc3_log.setLevel(2**20)

In [ ]:
# copied this here to allow for kernel restrating after prior cell
linreg_ftrs_p1 = np.c_[np.arange(10), np.ones(10)] # +1 trick in data

true_wgts  = m,b = w_1, w_0 = 3,2
linreg_tgt = rdot(true_wgts, linreg_ftrs_p1)

linreg_table = pd.DataFrame(linreg_ftrs_p1, 
                            columns=['ftr_1', 'ones'])
linreg_table['tgt'] = linreg_tgt
linreg_table[:3]

In [ ]:
# copied from other notebook
logreg_ftr = np.random.uniform(5,15, size=(100,))

true_wgts  = m,b = -2, 20
line_of_logodds = m*logreg_ftr + b
prob_at_x = np.exp(line_of_logodds) / (1 + np.exp(line_of_logodds))

logreg_tgt = np.random.binomial(1, prob_at_x, len(logreg_ftr))

logreg_ftrs_p1 = np.c_[logreg_ftr,
                       np.ones_like(logreg_ftr)]

logreg_table = pd.DataFrame(logreg_ftrs_p1, 
                            columns=['ftr_1','ones'])
logreg_table['tgt'] = logreg_tgt
display(logreg_table.head())

In [ ]:
with pm.Model() as model:
    # boilder plate-ish setup of the distributions of our 
    # guesses for things we don't know
    sd      = pm.HalfNormal('sd', sd=1)
    intercept  = pm.Normal('Intercept', 0, sd=20)
    ftr_1_wgt  = pm.Normal('ftr_1_wgt', 0, sd=20)

    # outcomes made from inital guess and input data
    # this is y = m x + b in an alternate form
    preds = ftr_1_wgt * linreg_table['ftr_1'] + intercept 

    # relationship between guesses, input data, and actual outputs
    # target = preds + noise(sd)  (noise == tolerance around the line)
    target = pm.Normal('tgt', 
                       mu=preds, sd=sd, 
                       observed=linreg_table['tgt'])

    linreg_trace = pm.sample(1000, progressbar=False)

In [ ]:
pm.summary(linreg_trace)[['mean']]

In [ ]:
with pm.Model() as model:
    pm.glm.GLM.from_formula('tgt ~ ftr_1', linreg_table,
                           family=pm.glm.families.Normal())
    linreg_trace = pm.sample(5000, progressbar=False)
pm.summary(linreg_trace)[['mean']]

In [ ]:
pm.traceplot(linreg_trace);

In [ ]:
with pm.Model() as model:
    pm.glm.GLM.from_formula('tgt ~ ftr_1', logreg_table, 
                            family=pm.glm.families.Binomial())
    logreg_trace = pm.sample(10000, progressbar=False)
pm.summary(logreg_trace)[['mean']]

In [ ]:
%matplotlib inline

df_trace = pm.trace_to_dataframe(logreg_trace)
sns.jointplot('ftr_1', 'Intercept', data=df_trace, 
              kind='kde', stat_func=None, height=4);